## Logistic Regression

Notebook with implementation of the Logistic Regression algorithm to predict victory in Dota 2

-------------------------------------------------------------------------------------------------------------------------------

#### If you are running this code on Google Colab, you need to first upload the following feature file: *dota2_regular_features.csv*

## Regular matches

Useful functions to use to explore the data and preprocessing steps before feeding the data into the algorithm:

* df.columns : to see the names of the columns (i.e., features)
* df.dtype : to see the types in the data
* data.head()
* data.info()
* df.describe()

In [ ]:
# Import necessary libraries
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score, roc_auc_score, roc_curve
from sklearn import metrics
import statistics as st
from sklearn import preprocessing

In [ ]:
# NOTE: uncomment this cell if you are running this code on a local machine. Please adjust the following variables to correctly point to the feature file location on your machine

# # Set directory for the regular match group
# cwd = os.getcwd()
# root_directory = os.path.dirname(cwd)

# regular_data_dir = root_directory + "\\model_features_pre-match\\regular\\"
# path_to_features = regular_data_dir + "dota2_regular_features.csv"

In [ ]:
# NOTE: use this cell if you are running this code on Google Colab

# Set directory for the regular match group. Make sure the feature file is uploaded to this Colab session
path_to_features = "/content/dota2_regular_features.csv"

In [ ]:
# Read the data (model feature file)
feature_regular_df = pd.read_csv(path_to_features)

### Exploration and preprocessing of the data

In [ ]:
# Drop first column (match id)
feature_regular_df = feature_regular_df.drop(['match_id'], axis=1)

# Print feature names
feature_regular_df.columns

In [ ]:
feature_regular_df.head()

In [ ]:
feature_regular_df.info()

In [ ]:
#Fill in missing data with the median value of the feature
feature_regular_df = feature_regular_df.fillna(feature_regular_df.median())

In [ ]:
feature_regular_df['rad_first_pick'] = feature_regular_df['rad_first_pick'].astype(int)

### Model building, training and evaluation

In [ ]:
# Import logistic regression library
from sklearn.linear_model import LogisticRegression

In [ ]:
# Split into features (X) and label (y)
X, y = feature_regular_df.iloc[:,:-1],feature_regular_df.iloc[:,-1]

In [ ]:
features = [c for c in feature_regular_df.columns if c != 'win_label']
target = 'win_label'

In [ ]:
# Define the number of folds for the k-fold cross-validation
kfolds = KFold(n_splits=10, shuffle=True)

In [ ]:
# instantiate the model (using the default parameters)
logreg = LogisticRegression(solver = 'liblinear')

# Scale features
columns_to_scale = list(range(238,456))

In [ ]:
# NOTE: the training process might take a while to execute

auc = list()

mm_scaler = preprocessing.MinMaxScaler()

for train_idx, test_idx in kfolds.split(X):
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_test, y_test = X.iloc[test_idx], y.iloc[test_idx]  
    
    X_train.iloc[:,columns_to_scale] = mm_scaler.fit_transform(X_train.iloc[:,columns_to_scale])
    logreg.fit(X_train, y_train)
    
    X_test.iloc[:,columns_to_scale] = mm_scaler.transform(X_test.iloc[:,columns_to_scale])
    y_pred_proba =logreg.predict_proba(X_test)[::,1]
    auc.append(metrics.roc_auc_score(y_test, y_pred_proba))

'Median AUC: {:.04f}'.format(st.median(auc))

C:\Users\markos-ece\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\markos-ece\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\markos-ece\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\markos-ece\Anaconda3\lib\site-p

C:\Users\markos-ece\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\markos-ece\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\markos-ece\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\markos-ece\Anaconda3\lib\site-p

'Median AUC: 0.6135'